In [6]:
import requests
import mysql.connector

#API configuration
API_URL = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json"

API_KEY = "UdfDtENKaaWlf1UzBnzVSldMxYu4umMwr3l1aOLd"

#Data base configuration
DB_CONFIG = {
    "host": "localhost",    
    "user": "root",         
    "password": "",         
    "database": "compl"    
}

def fetch_venues():
    """Fetch venue data from Sportradar API."""
    response = requests.get(API_URL, params={"api_key": API_KEY})

    if response.status_code == 200:
        complexes = response.json().get("complexes", [])
        venues = []

        for complex_data in complexes:
            complex_id = complex_data["id"]
            for venue in complex_data.get("venues", []):
                venues.append((
                    venue["id"],
                    venue["name"],
                    venue.get("city_name", None),
                    venue.get("country_name", None),
                    venue.get("timezone", None),
                    complex_id
                ))

        return venues
    else:
        print("Error fetching data:", response.status_code, response.text)
        return []

def insert_into_db(venues):
    """Insert venue data into MySQL."""
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()

        query = """
        INSERT INTO venues (venue_id, venue_name, city_name, country_name, timezone, complex_id) 
        VALUES (%s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            venue_name = VALUES(venue_name),
            city_name = VALUES(city_name),
            country_name = VALUES(country_name),
            timezone = VALUES(timezone),
            complex_id = VALUES(complex_id)
        """

        cursor.executemany(query, venues)
        conn.commit()

        print(f"{cursor.rowcount} venue records inserted/updated.")

        cursor.close()
        conn.close()

    except mysql.connector.Error as err:
        print("Database error:", err)

if __name__ == "__main__":
    venues = fetch_venues()
    if venues:
        insert_into_db(venues)
    else:
        print("No venue data to insert.")


3212 venue records inserted/updated.
